In [1]:
# Needed to switch directory in Google drive so as to import MARL env.
from google.colab import drive 
drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
!pwd
!pip install -r requirements.txt
!pip show tensorflow
!pip show ray

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
Obtaining gym_continuousDoubleAuction from git+https://github.com/ChuaCheowHuan/gym-continuousDoubleAuction.git@c897137cbcc93ca71cbd51c27e683c3298f6562d#egg=gym_continuousDoubleAuction (from -r requirements.txt (line 24))
  Skipping because already up-to-date.
  Found existing installation: gym-continuousDoubleAuction 0.0.1
    Can't uninstall 'gym-continuousDoubleAuction'. No files were found to uninstall.
  Running setup.py develop for gym-continuousDoubleAuction
Name: tensorflow
Version: 2.1.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"

import argparse
import gym
import random
import numpy as np

import ray
from ray import tune
from ray.rllib.utils import try_import_tf
from ray.tune.registry import register_env
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet_v2 import FullyConnectedNetwork
from ray.rllib.models import Model, ModelCatalog
from ray.rllib.policy.policy import Policy
from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy
from ray.tune.logger import pretty_print


import sys
if "../" not in sys.path:
    sys.path.append("../")

from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv

tf = try_import_tf()

In [0]:
class CustomModel_1(Model):
    def _lstm(self, Inputs, cell_size):
        s = tf.expand_dims(Inputs, axis=1, name='time_major')  # [time_step, feature] => [time_step, batch, feature]
        lstm_cell = tf.nn.rnn_cell.LSTMCell(cell_size)
        self.init_state = lstm_cell.zero_state(batch_size=1, dtype=tf.float32)
        # time_major means [time_step, batch, feature] while batch major means [batch, time_step, feature]
        outputs, self.final_state = tf.nn.dynamic_rnn(cell=lstm_cell, inputs=s, initial_state=self.init_state, time_major=True)
        lstm_out = tf.reshape(outputs, [-1, cell_size], name='flatten_rnn_outputs')  # joined state representation
        return lstm_out

    def _build_layers_v2(self, input_dict, num_outputs, options):
        hidden = 512
        cell_size = 256
        #S = input_dict["obs"]
        S = tf.layers.flatten(input_dict["obs"])
        with tf.variable_scope(tf.VariableScope(tf.AUTO_REUSE, "shared"),
                               reuse=tf.AUTO_REUSE,
                               auxiliary_name_scope=False):
            last_layer = tf.layers.dense(S, hidden, activation=tf.nn.relu, name="fc1")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc2")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc3")

        last_layer = self._lstm(last_layer, cell_size)

        output = tf.layers.dense(last_layer, num_outputs, activation=tf.nn.softmax, name="mu")

        return output, last_layer

In [0]:
def make_RandomPolicy(_seed):

    # a hand-coded policy that acts at random in the env (doesn't learn)
    class RandomPolicy(Policy):
        """Hand-coded policy that returns random actions."""
        def __init__(self, observation_space, action_space, config):
            self.observation_space = observation_space
            self.action_space = action_space
            self.action_space.seed(_seed)

        def compute_actions(self,
                            obs_batch,
                            state_batches,
                            prev_action_batch=None,
                            prev_reward_batch=None,
                            info_batch=None,
                            episodes=None,
                            **kwargs):
            """Compute actions on a batch of observations."""
            return [self.action_space.sample() for _ in obs_batch], [], {}

        def learn_on_batch(self, samples):
            """No learning."""
            #return {}
            pass

        def get_weights(self):
            pass

        def set_weights(self, weights):
            pass

    return RandomPolicy

In [5]:
ray.init(ignore_reinit_error=True, log_to_driver=False, webui_host='127.0.0.1', num_cpus=2)

2020-03-19 07:55:30,680	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-19 07:55:30,681	INFO resource_spec.py:212 -- Starting Ray with 6.74 GiB memory available for workers and up to 3.37 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-19 07:55:31,198	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2020-03-19_07-55-30_679121_5591/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-19_07-55-30_679121_5591/sockets/raylet',
 'redis_address': '172.28.0.2:38993',
 'session_dir': '/tmp/ray/session_2020-03-19_07-55-30_679121_5591',
 'webui_url': '127.0.0.1:8265'}

In [0]:
num_agents = 4
num_policies = num_agents
num_iters = 3
simple = False #store_true
num_of_traders = num_agents
tape_display_length = 10 
tick_size = 1
init_cash = 1000000
max_step = 700 # per episode 
episode = 5 

In [7]:
single_CDA_env = continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step)
obs_space = single_CDA_env.observation_space
act_space = single_CDA_env.action_space
register_env("continuousDoubleAuction-v0", lambda _: continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step))
ModelCatalog.register_custom_model("model_disc", CustomModel_1)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
# Each policy can have a different configuration (including custom model)
def gen_policy(i):
    config = {"model": {"custom_model": "model_disc"},
              "gamma": 0.99,}
    return (None, obs_space, act_space, config)

In [0]:
def policy_mapper(agent_id):
    for i in range(num_agents):
        if agent_id == i:
            return "policy_{}".format(i)

In [0]:
# Dictionary of policies
policies = {"policy_{}".format(i): gen_policy(i) for i in range(num_policies)}

In [11]:
# override policy with random policy

def set_RandomPolicy(policies):
    """
    Set 1st policy as PPO & override all other policies as RandomPolicy with
    different seed.
    """

    for i in range(num_agents):
        if i == num_agents-1:
            break
        x = i + 1
        policies["policy_{}".format(num_policies-x)] = (make_RandomPolicy(num_policies-x), obs_space, act_space, {})

    print('policies:', policies)
    return 0

set_RandomPolicy(policies)

policy_ids = list(policies.keys())

policies: {'policy_0': (None, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_1': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {}), 'policy_2': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {}), 'policy_3': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {})}


In [12]:
def main(args):    
    config = ppo.DEFAULT_CONFIG.copy()
    config["log_level"] = "DEBUG"
    config["multiagent"] = {"policies_to_train": ["policy_0"],
                            "policies": policies,
                            "policy_mapping_fn": policy_mapper,
                           }
    config["num_workers"] = 1
    config["num_envs_per_worker"] = 4   
    config["train_batch_size"] = 128
    config["batch_mode"] = "complete_episodes"
    config["sample_batch_size"] = 32

    local_dir="/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/chkpt/" # dir where your chkpts are saved
    chkpt = 59 # set the restore chkpt
    restore_path = "{}checkpoint_{}/checkpoint-{}".format(local_dir, chkpt, chkpt)

    trainer = ppo.PPOTrainer(config=config, 
                             env="continuousDoubleAuction-v0")    
    #trainer.restore(restore_path) # uncomment this to restore from chkpt

    for i in range(30):
        # Perform one iteration of training the policy with PPO
        result = trainer.train()
        print(pretty_print(result))
     
        if i % 3 == 0:
            checkpoint = trainer.save(local_dir)
            print("checkpoint saved at", checkpoint)

    checkpoint = trainer.save(local_dir)
    print("checkpoint saved at", checkpoint)


# run main
main(None)            

2020-03-19 07:55:32,298	INFO trainer.py:420 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-03-19 07:55:32,477	INFO ppo.py:165 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-03-19 07:55:32,539	DEBUG worker_set.py:179 -- Creating TF session {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}
2020-03-19 07:55:32,619	DEBUG rollout_worker.py:791 -- Creating policy for policy_0
2020-03-19 07:55:32,640	DEBUG catalog.py:406 -- Created preprocessor <ray.rllib.models.preprocessors.NoPreprocessor object at

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 5)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 5)


2020-03-19 07:55:35,484	DEBUG catalog.py:532 -- Using custom model model_disc
2020-03-19 07:55:35,594	DEBUG catalog.py:522 -- Created model <__main__.CustomModel_1 object at 0x7fdab6c5dda0>: ({'obs': <tf.Tensor 'policy_0/observation:0' shape=(?, 4, 10) dtype=float32>, 'prev_actions': <tf.Tensor 'policy_0/prev_action:0' shape=(?, 5) dtype=float32>, 'prev_rewards': <tf.Tensor 'policy_0/prev_reward:0' shape=(?,) dtype=float32>, 'is_training': <tf.Tensor 'policy_0/is_training:0' shape=() dtype=bool>} of Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), None, None) -> Tensor("policy_0/default_model_2/value_function/mu/Softmax:0", shape=(?, 1), dtype=float32), []
2020-03-19 07:55:35,671	DEBUG catalog.py:532 -- Using custom model model_disc
2020-03-19 07:55:35,756	DEBUG catalog.py:522 -- Created model <__main__.CustomModel_1 object at 0x7fdab6bac5f8>: ({'obs': <tf.Tensor 'policy_0/packed:0' shape=(1, 4, 10) dtype=float32>, 'prev_actions': <tf.Tensor 'policy_0/packed

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 5)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 5)


2020-03-19 07:55:38,673	DEBUG catalog.py:522 -- Created model <__main__.CustomModel_1 object at 0x7fdab5c8d3c8>: ({'obs': <tf.Tensor 'policy_0/observation:0' shape=(?, 4, 10) dtype=float32>, 'prev_actions': <tf.Tensor 'policy_0/prev_action:0' shape=(?, 5) dtype=float32>, 'prev_rewards': <tf.Tensor 'policy_0/prev_reward:0' shape=(?,) dtype=float32>, 'is_training': <tf.Tensor 'policy_0_1/tower/is_training:0' shape=() dtype=bool>} of Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), None, None) -> Tensor("policy_0_1/tower/default_model_1/value_function/mu/Softmax:0", shape=(?, 1), dtype=float32), []
2020-03-19 07:55:38,677	DEBUG catalog.py:532 -- Using custom model model_disc
2020-03-19 07:55:38,790	DEBUG catalog.py:522 -- Created model <__main__.CustomModel_1 object at 0x7fdab5da7c88>: ({'obs': <tf.Tensor 'policy_0/observation:0' shape=(?, 4, 10) dtype=float32>, 'is_training': True, 'prev_actions': <tf.Tensor 'policy_0/prev_action:0' shape=(?, 5) dtype=float32>

...

custom_metrics: {}
date: 2020-03-19_08-41-55
done: false
episode_len_mean: 701.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 1
episodes_total: 57
experiment_id: b6d1c176f8974af2bfd45911d6888178
hostname: 91380f02bba8
info:
  grad_time_ms: 79334.06
  learner:
    policy_0:
      cur_kl_coeff: 0.5062500238418579
      cur_lr: 4.999999873689376e-05
      entropy: 7.755210876464844
      entropy_coeff: 0.0
      kl: 0.012456420809030533
      policy_loss: -0.06198450177907944
      total_loss: 1244473.75
      vf_explained_var: 0.0
      vf_loss: 1244473.75
  load_time_ms: 14.561
  num_steps_sampled: 39957
  num_steps_trained: 36480
  sample_time_ms: 15389.408
  update_time_ms: 23.086
iterations_since_restore: 28
node_ip: 172.28.0.2
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 97.68434782608695
  ram_util_percent: 20.699999999999992
pid: 5591
policy_reward_max:
  policy_0: 128607.0
  policy_1: 77936.0
  policy_2:

2020-03-19 08:43:15,361	INFO multi_gpu_optimizer.py:143 -- Collected more training samples than expected (actual=701, train_batch_size=128). This may be because you have many workers or long episodes in 'complete_episodes' batch mode.
2020-03-19 08:43:15,362	INFO tf_policy.py:395 -- Optimizing variable <tf.Variable 'policy_0/default_model/fc2/kernel:0' shape=(512, 512) dtype=float32_ref>
2020-03-19 08:43:15,363	INFO tf_policy.py:395 -- Optimizing variable <tf.Variable 'policy_0/default_model/fc2/bias:0' shape=(512,) dtype=float32_ref>
2020-03-19 08:43:15,364	INFO tf_policy.py:395 -- Optimizing variable <tf.Variable 'policy_0/default_model/fc3/kernel:0' shape=(512, 512) dtype=float32_ref>
2020-03-19 08:43:15,365	INFO tf_policy.py:395 -- Optimizing variable <tf.Variable 'policy_0/default_model/fc3/bias:0' shape=(512,) dtype=float32_ref>
2020-03-19 08:43:15,366	INFO tf_policy.py:395 -- Optimizing variable <tf.Variable 'policy_0/default_model/rnn/lstm_cell/kernel:0' shape=(768, 1024) dtype

custom_metrics: {}
date: 2020-03-19_08-44-36
done: false
episode_len_mean: 701.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 1
episodes_total: 58
experiment_id: b6d1c176f8974af2bfd45911d6888178
hostname: 91380f02bba8
info:
  grad_time_ms: 79462.244
  learner:
    policy_0:
      cur_kl_coeff: 0.5062500238418579
      cur_lr: 4.999999873689376e-05
      entropy: 7.759871006011963
      entropy_coeff: 0.0
      kl: 0.008400293067097664
      policy_loss: -0.041761450469493866
      total_loss: 35709320.0
      vf_explained_var: 0.0
      vf_loss: 35709320.0
  load_time_ms: 14.16
  num_steps_sampled: 40658
  num_steps_trained: 37120
  sample_time_ms: 23349.068
  update_time_ms: 23.13
iterations_since_restore: 29
node_ip: 172.28.0.2
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 77.86260869565218
  ram_util_percent: 20.86739130434783
pid: 5591
policy_reward_max:
  policy_0: 128607.0
  policy_1: 77936.0
  policy_2: 

2020-03-19 08:44:39,538	DEBUG multi_gpu_optimizer.py:205 -- 0 {'cur_kl_coeff': 0.5062500238418579, 'cur_lr': 4.999999873689376e-05, 'total_loss': 20945772.0, 'policy_loss': -0.010612282, 'vf_loss': 20945772.0, 'vf_explained_var': 0.0, 'kl': 0.009538524, 'entropy': 7.758176, 'entropy_coeff': 0.0}
2020-03-19 08:44:42,217	DEBUG multi_gpu_optimizer.py:205 -- 1 {'cur_kl_coeff': 0.5062500238418579, 'cur_lr': 4.999999873689376e-05, 'total_loss': 20945772.0, 'policy_loss': -0.019643921, 'vf_loss': 20945772.0, 'vf_explained_var': 0.0, 'kl': 0.00422633, 'entropy': 7.740348, 'entropy_coeff': 0.0}
2020-03-19 08:44:44,876	DEBUG multi_gpu_optimizer.py:205 -- 2 {'cur_kl_coeff': 0.5062500238418579, 'cur_lr': 4.999999873689376e-05, 'total_loss': 20945772.0, 'policy_loss': -0.026252622, 'vf_loss': 20945772.0, 'vf_explained_var': 0.0, 'kl': 0.0025179698, 'entropy': 7.7299843, 'entropy_coeff': 0.0}
2020-03-19 08:44:47,570	DEBUG multi_gpu_optimizer.py:205 -- 3 {'cur_kl_coeff': 0.5062500238418579, 'cur_lr':

custom_metrics: {}
date: 2020-03-19_08-45-55
done: false
episode_len_mean: 701.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 1
episodes_total: 59
experiment_id: b6d1c176f8974af2bfd45911d6888178
hostname: 91380f02bba8
info:
  grad_time_ms: 79619.822
  learner:
    policy_0:
      cur_kl_coeff: 0.5062500238418579
      cur_lr: 4.999999873689376e-05
      entropy: 7.752533912658691
      entropy_coeff: 0.0
      kl: 0.009774331003427505
      policy_loss: -0.07509350031614304
      total_loss: 20945772.0
      vf_explained_var: 0.0
      vf_loss: 20945772.0
  load_time_ms: 14.566
  num_steps_sampled: 41359
  num_steps_trained: 37760
  sample_time_ms: 23349.058
  update_time_ms: 22.844
iterations_since_restore: 30
node_ip: 172.28.0.2
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 97.63893805309733
  ram_util_percent: 20.900000000000002
pid: 5591
policy_reward_max:
  policy_0: 128607.0
  policy_1: 77936.0
  policy_2